## LU Decomposition with Pivoting to Solve Ax = b

Instead of directly computing A⁻¹ or using costly matrix operations, we:

1. **Decompose A into L and U** with row pivoting:  
   \( PA = LU \)  
   where:
   - L = Lower triangular matrix with 1s on diagonal
   - U = Upper triangular matrix
   - P = Permutation matrix (row swaps)

2. **Solve using substitutions**:
   - Step 1: Solve \( Lz = Pb \) using **forward substitution**
   - Step 2: Solve \( Ux = z \) using **backward substitution**

This method is numerically more stable and avoids inverting matrices.


In [8]:
import numpy as np


In [9]:
def lu_decomposition_with_partial_pivoting(matrix_A):
    """
    Performs LU decomposition of matrix_A using Gaussian Elimination with Partial Pivoting.
    Returns:
        L: Lower triangular matrix
        U: Upper triangular matrix
        P: Permutation matrix (for row swaps)
    """
    number_of_rows = matrix_A.shape[0]
    permutation_matrix = np.eye(number_of_rows)
    lower_matrix = np.zeros_like(matrix_A)
    upper_matrix = matrix_A.copy().astype(float)

    for k in range(number_of_rows):
        # Partial Pivoting: Find index of max value in column k from row k to end
        max_row_index = np.argmax(np.abs(upper_matrix[k:, k])) + k

        if upper_matrix[max_row_index, k] == 0:
            raise ValueError("Matrix is singular!")

        # Swap rows in U
        upper_matrix[[k, max_row_index]] = upper_matrix[[max_row_index, k]]

        # Swap rows in permutation matrix
        permutation_matrix[[k, max_row_index]] = permutation_matrix[[max_row_index, k]]

        # Swap rows in lower matrix (only for the part that was filled)
        lower_matrix[[k, max_row_index], :k] = lower_matrix[[max_row_index, k], :k]

        # Elimination
        for i in range(k+1, number_of_rows):
            multiplier = upper_matrix[i, k] / upper_matrix[k, k]
            lower_matrix[i, k] = multiplier
            upper_matrix[i, :] = upper_matrix[i, :] - multiplier * upper_matrix[k, :]

    # Fill the diagonal of L with 1s
    np.fill_diagonal(lower_matrix, 1.0)

    return lower_matrix, upper_matrix, permutation_matrix


In [10]:
def forward_substitution(lower_matrix, vector_b):
    """
    Solves Lz = b using forward substitution.
    Returns the intermediate vector z.
    """
    number_of_rows = lower_matrix.shape[0]
    vector_z = np.zeros_like(vector_b)

    for i in range(number_of_rows):
        sum_terms = sum(lower_matrix[i, j] * vector_z[j] for j in range(i))
        vector_z[i] = (vector_b[i] - sum_terms) / lower_matrix[i, i]

    return vector_z


In [11]:
def backward_substitution(upper_matrix, vector_z):
    """
    Solves Ux = z using backward substitution.
    Returns the final solution vector x.
    """
    number_of_rows = upper_matrix.shape[0]
    vector_x = np.zeros_like(vector_z)

    for i in reversed(range(number_of_rows)):
        sum_terms = sum(upper_matrix[i, j] * vector_x[j] for j in range(i + 1, number_of_rows))
        vector_x[i] = (vector_z[i] - sum_terms) / upper_matrix[i, i]

    return vector_x


In [12]:
def solve_using_lu(matrix_A, vector_b):
    """
    Solves the system Ax = b using LU decomposition with pivoting,
    followed by forward and backward substitution.
    """
    lower_matrix, upper_matrix, permutation_matrix = lu_decomposition_with_partial_pivoting(matrix_A)
    print("Lower Matrix (L):\n", lower_matrix)
    print("Upper Matrix (U):\n", upper_matrix)
    # Apply permutation to b
    modified_vector_b = np.dot(permutation_matrix, vector_b)

    # Solve L * z = Pb using forward substitution
    vector_z = forward_substitution(lower_matrix, modified_vector_b)

    # Solve U * x = z using backward substitution
    solution_vector_x = backward_substitution(upper_matrix, vector_z)

    return solution_vector_x


In [13]:
# Define matrix A and vector b
matrix_A = np.array([
    [2.0, 3.0, 1.0],
    [4.0, 7.0, 7.0],
    [6.0, 18.0, 22.0]
])

vector_b = np.array([1.0, 2.0, 3.0])

# Solve Ax = b
solution_x = solve_using_lu(matrix_A, vector_b)
print("Solution vector x:")
print(solution_x)


Lower Matrix (L):
 [[1.         0.         0.        ]
 [0.66666667 1.         0.        ]
 [0.33333333 0.6        1.        ]]
Upper Matrix (U):
 [[ 6.         18.         22.        ]
 [ 0.         -5.         -7.66666667]
 [ 0.          0.         -1.73333333]]
Solution vector x:
[ 0.5 -0.  -0. ]


## Problem Statement - Material Balance Using LU Decomposition

We are given a separation process in a petrochemical industry where 150 kg/h of a mixture of propane (P), butane (B), and propylene (PP) enters the first distillation column. The compositions and molecular weights are:

- **Feed to Column 1:** 150 kg/h, P = 0.5, PP = 0.2
- **Molecular Weights:**  
  - Propane (P) = 44 g/mol  
  - Butane (B) = 58 g/mol  
  - Propylene (PP) = 42 g/mol

From the given table:

|               | Top Stream           | Bottom Stream        |
|---------------|----------------------|----------------------|
| **Column 1**  | P = 0.9, PP = 0.08    | P = 0.2, PP = 0.2     |
| **Column 2**  | P = 0.95, PP = 0.03   | P = 0.05, PP = 0.85   |

We need to find the **molar flow rates** (in mol/h) of the **four output streams**:
- `x1`: Top stream of column 1
- `x2`: Bottom stream of column 1 (feed to column 2)
- `x3`: Top stream of column 2
- `x4`: Bottom stream of column 2

We'll form material balance equations for **Propane**, **Propylene**, and **Total moles**, and solve them using **LU decomposition**.


In [14]:
# Given data
feed_kgph = 150  # kg/h
mol_weights = {
    'P': 44,    # Propane
    'B': 58,    # Butane
    'PP': 42    # Propylene
}

# Mole fractions in feed
feed_composition = {
    'P': 0.5,
    'PP': 0.2,
    'B': 1 - 0.5 - 0.2  # rest is Butane
}

# Calculate total molar flow rate (mol/h)
total_moles = sum((feed_composition[comp] * feed_kgph * 1000) / mol_weights[comp] for comp in mol_weights)
print(f"Total molar flow rate of feed: {total_moles:.2f} mol/h")


Total molar flow rate of feed: 3194.69 mol/h


### System of Linear Equations

Let:

- `x1`: Top stream of Column 1 (mol/h)
- `x2`: Bottom stream of Column 1 = Feed to Column 2 (mol/h)
- `x3`: Top stream of Column 2 (mol/h)
- `x4`: Bottom stream of Column 2 (mol/h)

Based on the mole balances of P and PP and total moles:

**Equation 1 (Propane balance):**  
0.5 * F = 0.9*x1 + 0.2*x2  
       = 0.9*x1 + 0.2*(x3 + x4)

**Equation 2 (Propylene balance):**  
0.2 * F = 0.08*x1 + 0.2*x2  
       = 0.08*x1 + 0.2*(x3 + x4)

**Equation 3 (Total molar balance):**  
F = x1 + x2 = x1 + x3 + x4

We simplify these into the matrix form `Ax = b`


In [15]:
import numpy as np

F = total_moles  # mol/h

# Coefficients for matrix A
A = np.array([
    [0.9, 0.2, 0.2, 0.2],   # Propane balance
    [0.08, 0.2, 0.2, 0.2],  # Propylene balance
    [1.0, -1.0, -1.0, -1.0] # Total molar balance: x1 = x2 + x3 + x4
])

# Right-hand side vector b
b = np.array([
    0.5 * F,  # Propane input
    0.2 * F,  # Propylene input
    0         # x1 - x2 - x3 - x4 = 0
])


### Final Molar Flow Rates (mol/h)

From the LU decomposition solution, we get:

- `x1` = Top stream of Column 1  
- `x2` = Bottom stream of Column 1 / Feed to Column 2  
- `x3` = Top stream of Column 2  
- `x4` = Bottom stream of Column 2

These values represent the **molar flow rates** of each output stream.
You can multiply each with molar fractions to get the component-wise flow as well.
